HAI Card Game

In [27]:
import pandas as pd
import random
import numpy as np
from linear_irl import large_irl

raw_data = pd.read_csv('Card Game Responses.csv')
raw_data

,Timestamp,Paste your data below (preserving line breaks and spacing),Email Address
0,3/25/2023 23:17:35,4 6 131242412323142243443131\n0000001111112222...,edwardneo@berkeley.edu
1,3/28/2023 18:35:16,4 6 131242412323142243443131\n2111000301112223...,yifengwang@berkeley.edu
2,3/29/2023 16:19:37,4 6 131242412323142243443131\n1233012202232101...,austin0707@berkeley.edu
3,3/31/2023 9:54:37,4 6 131242412323142243443131\n1211101021220033...,alex20020805@berkeley.edu
4,4/2/2023 1:59:30,4 6 131242412323142243443131\n1211102022023203...,kim.nahyun@berkeley.edu
5,4/2/2023 10:11:46,4 6 131242412323142243443131\n0332102102031032...,kareena.wu@berkeley.edu
6,4/2/2023 19:48:01,4 6 131242412323142243443131\n2111012220022103...,anishsasanur@berkeley.edu
7,4/2/2023 21:18:57,4 6 131242412323142243443131\n2110022101231021...,estseng@berkeley.edu
8,4/2/2023 23:07:53,4 6 131242412323142243443131\n2133102221100232...,nickmelamed@berkeley.edu
9,4/3/2023 15:14:03,4 6 131242412323142243443131\n0000001111112222...,mamingyuan2001@berkeley.edu


In [2]:
mod_data = raw_data['Paste your data below (preserving line breaks and spacing)'].str.extract(r'(\d) (\d) (\d+)\n(\d+) (\d+) (\d)\n(\d) (\d) (\d+)\n(\d+) (\d+) (\d)')
mod_data[12] = mod_data[2].str[:6]
mod_data[13] = mod_data[2].str[6:12]
mod_data[14] = mod_data[2].str[12:18]
mod_data[15] = mod_data[2].str[18:]
mod_data[16] = mod_data[8].str[: 7]
mod_data[17] = mod_data[8].str[7:14]
mod_data[18] = mod_data[8].str[14:]
mod_data.head(5)
actual_data = pd.DataFrame()
for i in range(6):
   actual_data[i] = pd.concat([mod_data[i], mod_data[i + 6]])
actual_data[6] = pd.concat([mod_data[12], mod_data[16]])
actual_data[7] = pd.concat([mod_data[13], mod_data[17]])
actual_data[8] = pd.concat([mod_data[14], mod_data[18]])
actual_data[9] = pd.concat([mod_data[15], pd.Series([None] * 15)])
actual_data.drop([2], axis=1, inplace=True)
actual_data.reset_index(drop=True, inplace=True)
actual_data = actual_data[[0, 1, 6, 7, 8, 9, 3, 4, 5]]
actual_data.rename(columns={0: 'gameboard_height', 1: 'gameboard_width', 3: 'elimination_sequence', 4: 'deck_length', 5: 'max_deck_length', 6: 'row_1', 7: 'row_2', 8: 'row_3', 9: 'row_4'}, inplace=True)
actual_data.head(5)

,gameboard_height,gameboard_width,row_1,row_2,row_3,row_4,elimination_sequence,deck_length,max_deck_length
0,4,6,131242,412323,142243,443131,000000111111222222333333,123456753423423123453120,7
1,4,6,131242,412323,142243,443131,211100030111222303322033,123123123423120123123120,4
2,4,6,131242,412323,142243,443131,123301220223210130010133,123123423423123123120120,4
3,4,6,131242,412323,142243,443131,121110102122003333023320,123120123123120123123120,3
4,4,6,131242,412323,142243,443131,121110202202320303310331,123120123420120123120120,4


In [3]:
def get_states(cards, height, maxLength, cardsInHand, memo = {}):
    if cards == [0] * height:
        return {f'0 ' * height + str(maxLength)}
    returnedCards = set()
    if (tuple(cards), cardsInHand, maxLength) in memo:
        return memo[(tuple(cards), cardsInHand, maxLength)]    
    for i, card in enumerate(cards):
        if card != 0:
            nextCardsInHand = cardsInHand * 10 + card % 10
            if ''.join([i for i in str(nextCardsInHand) if str(nextCardsInHand).count(str(i)) != 3]):
                nextCardsInHand = int(''.join([i for i in str(nextCardsInHand) if str(nextCardsInHand).count(str(i)) != 3]))
            else:
                nextCardsInHand = 0
            nextMaxLength = max(maxLength, len(str(nextCardsInHand)) if nextCardsInHand != 0 else 0)
            nextCards = cards.copy()
            nextCards[i] //= 10
            returnedCards.add(' '.join([str(i) for i in nextCards]) + ' ' + str(nextMaxLength))
            returnedCards |= get_states(nextCards, height, nextMaxLength, nextCardsInHand, memo)
    memo[(tuple(cards), cardsInHand, maxLength)] = set(returnedCards)
    return set(returnedCards)

In [4]:
def get_next_state(state_str, action, initial_state):
    state = state_str.split()
    initial_state = [str(i) for i in initial_state]
    cards_in_hand = []
    for i in range(len(initial_state)):
        if initial_state[i][len(state[i]):] != '':
            cards_in_hand.append(initial_state[i][len(state[i]):])
    if ''.join(cards_in_hand) == '':
        cards_in_hand = 0
    else:
        cards_in_hand = ''.join(cards_in_hand)
        impossible_cards = [i for i in cards_in_hand if cards_in_hand.count(i) >= 3]
        while impossible_cards:
            impossible_card_amount = cards_in_hand.count(impossible_cards[0])
            final_card_amount = impossible_card_amount % 3
            cards_in_hand = [i for i in list(cards_in_hand) if i != impossible_cards[0]]
            cards_in_hand.extend([impossible_cards[0]] * final_card_amount)
            cards_in_hand = ''.join(cards_in_hand)
            impossible_cards = [i for i in cards_in_hand if cards_in_hand.count(i) >= 3]
        if cards_in_hand != '':
            cards_in_hand = int(cards_in_hand)
        else:
            cards_in_hand = 0
    state = [int(i) for i in state]
    row_to_mod = state[action - 1]
    if row_to_mod == 0:
        return None
    card_added_to_hand = row_to_mod % 10
    row_to_mod //= 10
    cards_in_hand = cards_in_hand * 10 + card_added_to_hand
    if ''.join([i for i in str(cards_in_hand) if str(cards_in_hand).count(str(i)) != 3]):
        cards_in_hand = int(''.join([i for i in str(cards_in_hand) if str(cards_in_hand).count(str(i)) != 3]))
    else:
        cards_in_hand = 0
    state[action - 1] = row_to_mod
    return ' '.join([str(i) for i in state[:len(state) - 1]]) + ' ' + str(max(len(str(cards_in_hand)), state[-1]))    

In [5]:
def get_transition_probabilities(states, num_actions, initial_state):
    returned_lst_1 = []
    for i in states:
        returned_lst_2 = []
        next_states = [get_next_state(i, action + 1, initial_state) for action in range(num_actions)]
        num_of_nones = sum([state == None for state in next_states])
        for j in range(num_actions):
            returned_lst_3 = []
            if next_states[j] == None:
                returned_lst_2.append([0] * len(states))
            else:
                for k in states:
                    if k == next_states[j]:
                        returned_lst_3.append((1 / (num_actions - num_of_nones - 1 + 8)) * 8)
                    elif k in next_states:
                        returned_lst_3.append(1 / (num_actions - num_of_nones - 1 + 8))
                    else:
                        returned_lst_3.append(0)
                returned_lst_2.append(returned_lst_3)
        returned_lst_1.append(returned_lst_2)
    return np.array(returned_lst_1)

In [6]:
def get_policy(states, num_actions, initial_state, elimination_sequence):
    state = " ".join([str(i) for i in initial_state]) + ' 0'
    policy_list = [random.randint(0, num_actions - 1) for _ in range(len(states))]
    elimination_sequence = [int(i) for i in elimination_sequence]
    for i in elimination_sequence:
        index = list(states).index(state)
        policy_list[index] = i
        state = get_next_state(state, i + 1, initial_state)
    return policy_list    

In [24]:
def get_values(feature_matrix, policy, num_states, transition_probabilities, discount_factor = 0.9, threshold = 0.1):
    values = []
    percentage = 0
    for dim in range(feature_matrix.shape[1]):
        reward = feature_matrix[:, dim]
        values.append(value(policy, num_states, transition_probabilities, reward, discount_factor, threshold))
        percentage += 1
        print(f'{percentage / feature_matrix.shape[1] * 100}%')
    return np.array(values)

In [16]:
def get_feature_matrix(states, num_diff_cards):
    lst = []
    for state in states:
        if state[:7] == '0 0 0 0':
            lst.append([(num_diff_cards * 2 - int(state[-1])) / (num_diff_cards * 2)])
        else:
            lst.append([0])
    return np.array(lst)

In [ ]:
def get_feature_matrix_simp(states):
    lst = []
    for i in range(len(states)):
        feature = np.zeros(len(states))
        feature[i] = 1
        lst.append(feature)
    return np.array(lst)

In [11]:
states = get_states([131242, 412323, 142243, 443131], 4, 0, 0)
states = states.union({'131242 412323 142243 443131 0'})
num_states = len(states)
actions = 4
transition = get_transition_probabilities(states, actions, [131242, 412323, 142243, 443131])
policy = get_policy(states, actions, [131242, 412323, 142243, 443131], '211100030111222303322033')


In [17]:
feature_matrix = get_feature_matrix_simp(states, 4)
feature_matrix

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [25]:
values = get_values(feature_matrix, policy, num_states, transition)

Iteration: 1000 Difference: 0
Iteration: 2000 Difference: 0.5
Iteration: 3000 Difference: 0.625
Iteration: 4000 Difference: 0.625
Iteration: 5000 Difference: 0.625
Iteration: 6000 Difference: 0.625
Iteration: 7000 Difference: 0.625
Iteration: 8000 Difference: 0.625
Iteration: 8416 Difference: 0.625
Iteration: 9000 Difference: 0.027000000000000003
Iteration: 10000 Difference: 0.3375
Iteration: 11000 Difference: 0.3375
Iteration: 12000 Difference: 0.3375
Iteration: 13000 Difference: 0.3375
Iteration: 14000 Difference: 0.3375
Iteration: 15000 Difference: 0.3375
Iteration: 16000 Difference: 0.44999999999999996
Iteration: 16832 Difference: 0.44999999999999996
Iteration: 17000 Difference: 0.0003690185950413224
Iteration: 18000 Difference: 0.216
Iteration: 19000 Difference: 0.216
Iteration: 20000 Difference: 0.216
Iteration: 21000 Difference: 0.216
Iteration: 22000 Difference: 0.216
Iteration: 23000 Difference: 0.216
Iteration: 24000 Difference: 0.216
Iteration: 25000 Difference: 0.216
Iterat